第二章 end to end machine learning

一、结构化问题

二、获取数据与预处理

三、选择和训练模型

（一）在训练集上测试和评估

In [7]:
import os
os.environ["OMP_NUM_THREADS"]="7"
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.datasets import fetch_california_housing
import numpy as np

1、线性模型

In [22]:
#线性模型
house=fetch_california_housing()
housing_=pd.DataFrame(house.data,columns=house.feature_names)
target=pd.DataFrame(house.target)
raw_housing["target"]=target
lin_reg = LinearRegression()
lin_reg.fit(np.array(housing_), np.array(target))

LinearRegression()

In [27]:
raw_housing["income_cat"] = np.ceil(raw_housing["MedInc"] / 1.5)#向上取整
raw_housing["income_cat"].where(raw_housing["income_cat"] < 5, 5.0, inplace=True)#减少分层类别
raw_housing.head()
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)#n_splits:只生成一个训练集和测试集
for train_index, test_index in split.split(raw_housing,raw_housing["income_cat"]):
 strat_train_set = raw_housing.loc[train_index]
 strat_test_set = raw_housing.loc[test_index]
print(strat_test_set["income_cat"].value_counts() / len(strat_test_set))
#去除income_cat，还原数据
for set_ in (strat_train_set, strat_test_set):
 set_.drop("income_cat", axis=1, inplace=True)
strat_test_set.head()
housing1=strat_test_set
housing=housing1.drop("target",axis=1)

income_cat
3.0    0.350533
2.0    0.318798
4.0    0.176357
5.0    0.114341
1.0    0.039971
Name: count, dtype: float64


In [28]:
some_data = housing.iloc[:5]
some_labels =housing1["target"].iloc[:5]
print("Predictions:", lin_reg.predict(some_data))
print("Labels:", list(some_labels))

Predictions: [[3.95704674]
 [3.01570284]
 [2.40227245]
 [1.93534367]
 [2.34893857]]
Labels: [5.00001, 1.625, 2.046, 1.597, 1.84]


c:\ProgramData\Miniconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [31]:
#计算均方误差
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing)
lin_mse = mean_squared_error(housing1["target"], housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

c:\ProgramData\Miniconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


0.7116396619238798

2、决策数模型

In [32]:
#决策树模型
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing, housing1["target"])

DecisionTreeRegressor()

In [34]:
#计算决策树模型的均方误差
housing_predictions = tree_reg.predict(housing)
tree_mse = mean_squared_error(housing1["target"], housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse#过度拟合

2.69477289740792e-16

3、随机森林模型

In [47]:
#随机森林模型
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing, housing1["target"])
housing_predictions=forest_reg.predict(housing)
forest_mse=mean_squared_error(housing1["target"],housing_predictions)
forest_rmse=np.sqrt(forest_mse)
forest_rmse

0.20650680182699074

第二章 end to end machine learning

一、结构化问题

二、获取数据与预处理

三、选择和训练模型

（一）在训练集上测试和评估

（二）使用交叉验证更好地评估

In [42]:
from sklearn.model_selection import cross_val_score
#决策树
scores = cross_val_score(tree_reg, housing,housing1["target"],scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
 print("Scores:", scores)
 print("Mean:", scores.mean())
 print("Standard deviation:", scores.std())
display_scores(tree_rmse_scores)

Scores: [0.80139731 0.87787601 0.81254153 0.7757707  0.72935176 0.77227208
 0.80884693 0.7494664  0.76075699 0.7502379 ]
Mean: 0.7838517605585286
Standard deviation: 0.04075723635902619


In [44]:
#线性拟合
lin_scores = cross_val_score(lin_reg, housing, housing1["target"],scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [0.70004837 0.84141319 0.71052271 0.68463389 0.68655235 0.69444244
 0.7098668  0.68489686 0.6755935  0.72164556]
Mean: 0.7109615665974477
Standard deviation: 0.04554598625619705


In [48]:
#随机森林
forest_scores = cross_val_score(forest_reg, housing, housing1["target"],scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)#训练集的得分依旧低于测试集，说明过度拟合的情况依然存在于随机森林模型

Scores: [0.54571391 0.57130065 0.54024028 0.55583545 0.52373504 0.55038754
 0.58452551 0.55755842 0.53142793 0.5760599 ]
Mean: 0.5536784641209571
Standard deviation: 0.018518264553155906


保存训练过的模型方便以后对比（超参数，交叉验证分数，预测值等）

In [ ]:
import joblib
joblib.dump(my_model, "my_model.pkl")
my_model_loaded = joblib.load("my_model.pkl")

第二章 end to end machine learning

一、结构化问题

二、获取数据与预处理

三、选择和训练模型

（一）在训练集上测试和评估

（二）使用交叉验证更好地评估

四、微调模型（优化超参数）

（一）网格状搜索

（二）随机搜索

（三）系综方法

（四）分析最好的模型和它们的错误

In [64]:
#网格状搜索
from sklearn.model_selection import GridSearchCV
param_grid = [{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,scoring='neg_mean_squared_error')
grid_search.fit(housing, housing1["target"])
grid_search.best_params_
grid_search.best_estimator_

RandomForestRegressor(max_features=2, n_estimators=30)

In [65]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.6637333416190467 {'max_features': 2, 'n_estimators': 3}
0.5781791384996546 {'max_features': 2, 'n_estimators': 10}
0.544648305731754 {'max_features': 2, 'n_estimators': 30}
0.6685559219685402 {'max_features': 4, 'n_estimators': 3}
0.5759965150535331 {'max_features': 4, 'n_estimators': 10}
0.5510481072918636 {'max_features': 4, 'n_estimators': 30}
0.6708546545735012 {'max_features': 6, 'n_estimators': 3}
0.5868660493659836 {'max_features': 6, 'n_estimators': 10}
0.5637419357284918 {'max_features': 6, 'n_estimators': 30}
0.6656067107435734 {'max_features': 8, 'n_estimators': 3}
0.5908113257069303 {'max_features': 8, 'n_estimators': 10}
0.562099315096079 {'max_features': 8, 'n_estimators': 30}
0.6398174888102095 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
0.5582267799494335 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
0.6308416571945943 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
0.5556371296513247 {'bootstrap': False, 'max_features': 3, 'n_e

In [66]:
#随机搜索
from sklearn.model_selection import RandomizedSearchCV 

In [69]:
#分析最好的模型和它们的错误
feature_importances = grid_search.best_estimator_.feature_importances_
attributes=housing.columns
sorted(zip(feature_importances, attributes), reverse=True)

[(0.3307113806806973, 'MedInc'),
 (0.13776494692917154, 'Latitude'),
 (0.13456682998698616, 'Longitude'),
 (0.12493552192076493, 'AveRooms'),
 (0.12413173009595622, 'AveOccup'),
 (0.056748521412451486, 'HouseAge'),
 (0.05045546754878993, 'AveBedrms'),
 (0.040685601425182474, 'Population')]

第二章 end to end machine learning

一、结构化问题

二、获取数据与预处理

三、选择和训练模型

（一）在训练集上测试和评估

（二）使用交叉验证更好地评估

四、微调模型（优化超参数）

（一）网格状搜索

（二）随机搜索

（三）系综方法

（四）分析最好的模型和它们的错误

五、在测试集上评估模型

启动、监控和维护你的系统

In [71]:
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("target", axis=1)
y_test = strat_test_set["target"].copy()
final_predictions = final_model.predict(X_test)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse) 
final_rmse

0.2145128977316376